In [2]:
%%javascript
$('<div id="toc"></div>').css({position: 'fixed', top: '120px', left: 0}).appendTo(document.body);
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js');


<IPython.core.display.Javascript object>

# Imports

In [3]:
import pandas as pd
import numpy as np
import io

# Data

In [13]:
df_article = pd.read_csv('./dataset/dataset_articles_0.csv')
df_match = pd.read_csv('./dataset/dataset_articles_0.csv'')
df_claim = pd.read_csv('/data/peskine/long_document_similarity/claim_dataset0.csv')

In [14]:
df_article

,articleId,chunkId,chunk
0,0,0,New York Schools To Stay Closed For Rest Of Ac...
1,0,1,Schools in America's coronavirus epicenter of ...
2,0,2,The decision comes as the state remains cautio...
3,0,3,"""We don't think it's possible to (open schools..."
4,0,4,The confirmation came after Mayor Bill de Blas...
...,...,...,...
321,19,321,"Asked whether Trump runs in 2024, Graham deadp..."
322,19,322,Politics professor John Pitney Jr of Claremont...
323,19,323,"""Supporters of president Trump vastly outnumbe..."
324,19,324,"Listen to the latest songs, only on JioSaavn.com"


In [15]:
df_match

,articleId,chunkId1,genericClaim1,specificClaim1,hybridClaim1,chunkId2,genericClaim2,specificClaim2,hybridClaim2
0,0,5,1270,1310,1350,6,1271,1311,1351
1,1,18,1272,1312,1352,17,1273,1313,1353
2,2,32,1274,1314,1354,28,1275,1315,1355
3,3,40,1276,1316,1356,41,1277,1317,1357
4,4,54,1278,1318,1358,55,1279,1319,1359
5,5,62,1280,1320,1360,67,1281,1321,1361
6,6,89,1282,1322,1362,85,1283,1323,1363
7,7,98,1284,1324,1364,98,1285,1325,1365
8,8,115,1286,1326,1366,124,1287,1327,1367
9,9,140,1288,1328,1368,133,1289,1329,1369


In [16]:
df_claim

,claimId,claim
0,0,Trump tweeted he won the presidential elections
1,1,"""The tax plan rolled out by @realDonaldTrump w..."
2,2,"President Donald Trump said, ""If you elect Joe..."
3,3,"""Donald Trump plans to step down as President ..."
4,4,"""Twitter caught leaking Donald Trumps private ..."
...,...,...
1385,1385,With the rising number of people with anosmia ...
1386,1386,Sweden prime minister wants the COVID economic...
1387,1387,Sweden is pushing the deal on coronavirus reco...
1388,1388,Donald Trump still has high popularity amongst...



# Model

In [317]:
from sentence_transformers import SentenceTransformer, util
SBmodel = SentenceTransformer('all-mpnet-base-v2') # best average performance model

In [318]:
#embedding claims
emb_claims = SBmodel.encode(all_claims,show_progress_bar=True, batch_size=128)
#embedding chunks
emb_chunks = SBmodel.encode(all_chunks,show_progress_bar=True, batch_size=128)
cos_sim = util.cos_sim(emb_chunks, emb_claims)

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

# Matching

## Average

In [567]:
ranks1 = []
ranks2 = []
for article_id in range (0, 20):
    chunk_ids = df_chunks[df_chunks['articleId']==article_id]['chunkId'].tolist()
    generic_target1 = df_match[df_match['articleId']==article_id]['hybridClaim1'].tolist()[0]
    generic_target2 = df_match[df_match['articleId']==article_id]['hybridClaim2'].tolist()[0]
    
    claims_similarity_scores = cos_sim[chunk_ids].sum(dim=0).numpy()
        
    claim_ids_sorted = claims_similarity_scores.argsort()[::-1].tolist()
    ranks1.append(claim_ids_sorted.index(generic_target1))
    ranks2.append(claim_ids_sorted.index(generic_target2))


In [568]:
ranks1, ranks2, sum(ranks1)/20, sum(ranks2)/20

([1, 2, 2, 2, 3, 4, 1, 1, 1, 2, 2, 3, 3, 1, 4, 2, 2, 0, 5, 65],
 [5, 1, 31, 1, 0, 0, 4, 2, 3, 0, 0, 2, 0, 0, 0, 0, 0, 3, 0, 2],
 5.3,
 2.7)

## Maximum

In [492]:
ranks1 = []
ranks2 = []
for article_id in range (0, 20):
    chunk_ids = df_chunks[df_chunks['articleId']==article_id]['chunkId'].tolist()
    generic_target1 = df_match[df_match['articleId']==article_id]['hybridClaim1'].tolist()[0]
    generic_target2 = df_match[df_match['articleId']==article_id]['hybridClaim2'].tolist()[0]
    
    claims_similarity_scores = cos_sim[chunk_ids].max(dim=0)[0].numpy()
        
    claim_ids_sorted = claims_similarity_scores.argsort()[::-1].tolist()
    ranks1.append(claim_ids_sorted.index(generic_target1))
    ranks2.append(claim_ids_sorted.index(generic_target2))
    

In [493]:
ranks1, ranks2, sum(ranks1)/20, sum(ranks2)/20

([0, 1, 7, 1, 2, 7, 5, 0, 0, 5, 3, 4, 4, 1, 11, 2, 0, 2, 5, 6],
 [4, 0, 1, 8, 1, 1, 6, 2, 9, 4, 2, 6, 0, 2, 0, 5, 2, 8, 2, 1],
 3.3,
 3.2)

## Relevant chunk

In [504]:
ranks1 = []
ranks2 = []
for article_id in range (0, 20):
    chunk_id1 = df_match[df_match['articleId']==article_id]['chunkId1'].tolist()[0]
    chunk_id2 = df_match[df_match['articleId']==article_id]['chunkId2'].tolist()[0]
    generic_target1 = df_match[df_match['articleId']==article_id]['specificClaim1'].tolist()[0]
    generic_target2 = df_match[df_match['articleId']==article_id]['specificClaim2'].tolist()[0]
    
    claims_similarity_scores = cos_sim[chunk_id1].numpy()
    claim_ids_sorted = claims_similarity_scores.argsort()[::-1].tolist()
    ranks1.append(claim_ids_sorted.index(generic_target1))
    
    claims_similarity_scores = cos_sim[chunk_id2].numpy()
    claim_ids_sorted = claims_similarity_scores.argsort()[::-1].tolist()

    ranks2.append(claim_ids_sorted.index(generic_target2))
    

In [505]:
ranks1, ranks2, sum(ranks1)/20, sum(ranks2)/20

([0, 0, 2, 0, 4, 1, 0, 4, 3, 11, 0, 0, 0, 4, 226, 3, 0, 0, 0, 2],
 [66, 2, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 131],
 13.0,
 10.3)